# TEMP
作为临时查看用，实际处理都要复制到python文件，使得input为原始的jsonl格式

In [2]:

# 展示ollama模型信息，已经部署到llm.py中
import subprocess
import json

# 定义curl命令
command = ['curl', 'http://localhost:11434/api/ps']

# 执行命令
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# 获取标准输出并去掉多余的引号
output_str = result.stdout

# 解析为Python字典
json_data = json.loads(output_str)
# json_data['models'][0]['name']
# json_data

In [3]:
import pandas as pd
med = []
with open('0604_original_test.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        record = json.loads(line.strip())
        med.append(record)
med = pd.DataFrame(med)
med

,question,answer,options,meta_info,answer_idx,evidence
0,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者游文胜，男，50岁，主因“乙肝表面抗...,原发性肝癌,"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[C],NaN
1,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者孙国庆，男，61岁，主因“间断腹痛腹...,原发性肝癌,"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[C],NaN
2,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者郭佃峰，男，主因“巩膜黄染，确诊原发...,原发性肝癌,"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[C],NaN
3,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者30余年前体检发现肝功能异常，ALT...,原发性肝癌,"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[C],NaN
4,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者6天前（2021-11-24）因“发...,原发性肝癌,"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[C],NaN
...,...,...,...,...,...,...
195,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者中年女性，慢性病程，急性发作；9年前...,[脂肪肝],"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[A],患者中年女性，慢性病程，急性发作；9年前体检发现肝功能异常，转氨酶轻度升高，转肽酶升高明显。...
196,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者老年女性，慢性病程；13年前体检发现...,"[脂肪肝, 自身免疫性肝炎]","{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,"[A, E]","老年女性患者，慢性病程，体检发现肝功能异常,以转氨酶轻度升高为主，IgG、抗核抗体升高，腹部..."
197,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者主因原位肝移植术后5年入院。入院查体...,[脂肪肝],"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[A],1.该患者为中年男性，慢性病程\n2.原位肝移植术后5年\n3.既往：高血压4年余，血压最高...
198,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者主因“反复呕血、黑便3年余，TIPS...,[脂肪肝],"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[A],患者青年男性，慢性病程；临床以消化道出血起病，进一步胃镜、影像学检查发现肝硬化合并食管胃底静...


In [4]:
import joblib
import numpy as np


hotpot = joblib.load('../data/hotpot-qa-distractor-sample.joblib').reset_index(drop = True)
for ind, row in hotpot.iterrows():
    supporting_articles = row['supporting_facts']['title']
    articles = row['context']['title']
    sentences = row['context']['sentences'] 
    supporting_paragraphs = []
    for article in supporting_articles:
        supporting_paragraph = ''.join(sentences[np.where(articles == article)][0])
        supporting_paragraphs.append(supporting_paragraph)
    supporting_paragraphs = '\n\n'.join(supporting_paragraphs)
    hotpot.at[ind, 'supporting_paragraphs'] = supporting_paragraphs
display(hotpot['supporting_paragraphs'][3])

'Creature Comforts is a stop motion clay animation comedy mockumentary franchise originating in a 1989 British humorous animated short film of the same name. The film matched animated zoo animals with a soundtrack of people talking about their homes, making it appear as if the animals were being interviewed about their living conditions. It was created by Nick Park and Aardman Animations. The film later became the basis of a series of television advertisements for the electricity boards in the United Kingdom, and in 2003, a television series in the same style was released. An American version of the series was also made.\n\nCreature Comforts is a stop motion clay animation comedy mockumentary franchise originating in a 1989 British humorous animated short film of the same name. The film matched animated zoo animals with a soundtrack of people talking about their homes, making it appear as if the animals were being interviewed about their living conditions. It was created by Nick Park a

In [5]:
def process_answer(answer):
    if isinstance(answer,str): return answer
    elif isinstance(answer,list):
        if len(answer)==1: return answer[0]
        else: return '+'.join(answer)
    else: print(answer)
Q =  "请使用你的已有知识，判断该病人是以下这7个病中的哪一种或多种疾病。脂肪肝，药物性肝损伤，原发性肝癌，酒精性肝病，自身免疫性肝炎，肝囊肿，肝血管瘤。如果病人是多种疾病，用'+'号连接。"
for ind, row in med.iterrows():
    raw_q = row['question']
    context = raw_q.replace('请判断该病人是以下哪种疾病。病人检查如下：','') # 反整理出病情context
    med.at[ind,'context'] = context 
    # TODO:对齐195，199的evidence的提取
    med.at[ind,'QAonly'] = Q+'||'+context
    med.at[ind,'Q'] = Q
    med.at[ind,'process_answer'] = process_answer(row['answer'])
    # TODO: 如果用context模式，要设计COT引导的sample（看代码）
    

## 寻找MEDCOT示例

In [6]:
med.index[med['evidence'].notna()]

Int64Index([ 90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
            103, 104, 105, 106, 107, 109, 110, 111, 112, 113, 114, 116, 117,
            118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
            131, 132, 133, 134, 135, 166, 167, 168, 169, 170, 171, 172, 173,
            174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187,
            188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199],
           dtype='int64')

In [19]:
have_evidence_ind = med.index[med['evidence'].notna()]
# 计算上下文长度（字数）
med['context_length'] = med['context'].str.len()

# 根据索引过滤出非空evidence行的上下文长度
filtered_med = med.loc[have_evidence_ind]

# 按照context_length进行排序
sorted_med = filtered_med.sort_values(by='context_length')

# 打印排序结果
print(sorted_med[['context', 'context_length']])
# np.median(have_evidence_ind) 113

                                               context  context_length
96   入院情况：1.老年男性，慢性病程；2.半年前无明显诱因出现上腹部隐痛不适，阵发性，可耐受，时...             898
191  入院情况：患者主因“发现肝功异常3年”入院。患者肝功异常主要以ALT、GGT升高为主，保肝治...             953
120  入院情况：患者主因“转氨酶间断升高8月”入院，查体：腹部平坦，呼吸运动正常，无脐疝、腹壁静脉...             986
193  入院情况：患者于2月前因牙齿发炎、疼痛，就诊于口腔科门诊，服用药物治疗（具体不详），后出现腹...            1055
173  入院情况：主因“脂肪肝5年，阵发头晕1月余”入院。中年女性，慢性病程。5年前体检发现脂肪肝，...            1082
..                                                 ...             ...
135  入院情况：患者主因“间断皮肤、巩膜黄染1年，肝功能反复异常半年”入院。1年前患者以皮肤、巩膜...            3231
134  入院情况：患者1年前因全身乏力血糖升高于当地泰安医院入院调整血糖，入院期间发现肝功异常，行上...            3239
197  入院情况：患者主因原位肝移植术后5年入院。入院查体： 体温: 36.3℃ 脉搏: 116次/...            3384
167  入院情况：患者主因“发现肝功能异常1年半余，间断上腹痛3月余。”入院。查体：T：36.3℃ ...            3767
113  入院情况：患者青年女性，慢性病程；反复肝功异常14月余，以转氨酶和胆红素升高为主，ANA最高...            4326

[77 rows x 2 columns]


In [16]:
def fewshot_generator(row:int):
    # print(f'{row=}')
    print(f"Relevant Context: {med.loc[row,'context']}\nQuestion: {med.loc[row,'Q']}\nThought: {med.loc[row,'evidence']}\nAction: Finish[{med.loc[row,'process_answer']}]")
fewshot_generator(96)


Relevant Context: 入院情况：1.老年男性，慢性病程；2.半年前无明显诱因出现上腹部隐痛不适，阵发性，可耐受，时有胀痛，有恶心，未吐，伴有大便习惯及性状改变，表现为3-7天排便一次，时有腹泻、便秘交替进行，无粘液脓血便，无呕血、黑便，无发热，曾行腹部超声检查提示脂肪肝，胆囊腺肌症，为求进一步治疗，收入我科。 主诉：间断上腹部疼痛半年。 现病史：患者半年前无明显诱因出现上腹部隐痛不适，阵发性，可耐受，时有胀痛，有恶心，未吐，伴有大便习惯及性状改变，表现为3-7天排便一次，时有腹泻、便秘交替进行，无粘液脓血便，无呕血、黑便，无发热，曾行腹部超声检查提示脂肪肝，胆囊腺肌症，为求进一步治疗，收入我科。病来精神状态可，无消瘦、乏力，无排尿正常，体重无增减。 既往史: 否认高血压、心脏病史，否认糖尿病、脑血管病、精神疾病史。30余年前肺结核病史，已治愈。否认肝炎史、疟疾史。手术史：有，于2018年04月28日因中耳炎全麻下行左侧完壁式乳突根治+后鼓室开放+鼓室成型术（II型）。过敏史：无，输血史：无，预防接种史：无，传染病史：无。其他系统回顾无特殊。 个人史：出生原籍，久居于本地，否认疫水、疫区接触史，否认其他放射性物质及毒物接触史。免疫接种史不详。否认吸烟史，有饮酒史，否认去过疫区、否认接触过从疫区来的人。饮酒史40余年，42-56°白酒3两/天；未戒酒。 家族史：父母已逝，兄弟姐妹9人，否认家族中类似病史、传染病史、遗传病史及肿瘤病史。 检查分类: 超声 检查项目名称: 病房血管彩超 检查部位:病房颈动脉彩超 检查所见: 颈动脉:  双侧颈总动脉、颈内动脉、颈外动脉可视段管径对称，内-中膜不均匀增厚，左侧颈总动脉分叉处后壁见大小约13.9x4.0mm等回声斑块，管腔未见明显狭窄和局限性扩张，管腔内可见血流充盈，斑块处血流充盈缺损，血流速度正常。 检查结论: 双侧颈动脉内-中膜不均匀增厚伴左侧斑块形成 生化检查: 标本质量-静脉血(空腹)、总前列腺特异性抗原(TPSA)-静脉血(空腹)：4.43ng/mL、备注-静脉血(空腹)、游离前列腺特异性抗原(FPSA)-静脉血(空腹)：0.73ng/mL
Question: 请使用你的已有知识，判断该病人是以下这7个病中的哪一种或多种疾病。脂肪肝，药物性肝损伤，原发性肝癌，酒精性肝病，自身免疫性肝炎，肝囊肿，肝血管瘤。

In [17]:
fewshot_generator(130)


Relevant Context: 入院情况：患者老年女性，慢性病程；主要临床表现为乏力、食欲减退及恶心，查肝生化示ALT、AST、GGT 明显升高，ALB降低，IgG升高，ANA、SMA阳性，于我院行肝脏穿刺等检查诊断“自身免疫性肝炎”明确，规律服用激素+硫唑嘌呤，目前激素已停用，硫唑嘌呤维持治疗。2年前发现肿瘤标记物异常，胃镜、结肠镜检查考虑“胃息肉（山田I型），结肠多发息肉（山田I-IV型）”。此次为复查入院。 主诉：间断乏力6年，发现肿瘤标记物异常2年。 现病史：患者6年前因乏力、食欲减退及恶心就诊于我科，发现肝功能异常，查肝生化示ALT、AST、GGT 明显升高，ALB降低，IgG 1870mg/dl，ANA+1:320(均质斑点)+1：160（肌动蛋白）；SMA+1：160，行肝脏穿刺活检，诊断为药物诱导的AIH（曾应用希刻劳等抗生素），服用醋酸泼尼松片（初为30mg/d，并逐渐规律减量）及硫唑嘌呤片（50mg/d）治疗，并于门诊规律复查调整用药剂量。3年前（2017年4月）因血清生化学指标恢复满意停用硫唑嘌呤与醋酸泼尼松龙，于2017年8月复查肝功能及IgG示ALT、AST、GGT、IgG再次升高，遂重新服用醋酸泼尼松片（初为30mg/d，并逐渐规律减量）及硫唑嘌呤片（50mg/d）治疗，规律复查肝功能及免疫球蛋白，指标恢复良好，激素逐渐减量，2月前停用激素，继续服用硫唑嘌呤至今。2年前复查时发现肿瘤标记物异常（CA199 44.3U/mL，CA-50 37.56 U/mL），行胃镜、结肠镜检查示胃息肉（山田I型），结肠多发息肉（山田I-IV型），并行EMR术。术后无腹痛、便血，无发热，无心悸、头晕、心慌等不适。1年前复查胃镜、结肠镜，行胃肠息肉钳除术并留取病理，病理结果提示：胃底腺胃粘膜组织，呈活动性慢性炎，伴小凹上皮增生；幽门腺胃粘膜组织，呈轻度慢性萎缩性炎，伴轻度肠上皮化生；息肉状结肠粘膜组织呈慢性炎。近期患者仍有间断乏力、腹胀、反酸、纳差，无黄疸，无鼻衄及齿龈出血，无皮肤瘙痒及白陶土色变，无明显关节疼痛、口干眼干及脱发，现为复查及进一步诊治收入我科。患者自发病以来，精神状态可，食欲不佳，睡眠差，夜间尿频，每晚排尿约3-4次，大便不成形，每日排便约2-3次，体重下降10余斤。 既往史: 高血压病史12年，血压最高达180/100mmHg，初口服代

In [18]:
fewshot_generator(113)

Relevant Context: 入院情况：患者青年女性，慢性病程；反复肝功异常14月余，以转氨酶和胆红素升高为主，ANA最高1:1000，IgG最高5480mg/L；既往应用可致肝损伤药物（中药、抗生素等），曾多次激素治疗后好转，减量（4mg/d）后肝功异常反复加重；腹部超声提示肝脏高回声结节，考虑血管瘤，胆囊炎，脾大，腹水。BMI 22.6kg/m2 主诉：间断皮肤、巩膜黄染14月余 现病史：患者14月余前受凉出现寒战、发热，体温最高39.8℃，夜间明显，无咳嗽、咳痰，无腹痛、腹胀，无便秘、腹泻，无呕血、黑便，无陶土样大便，就诊于当地诊所予“连花清瘟颗粒、感冒清热颗粒、感冒疏风丸、布洛芬”等药物（具体剂量不详），治疗6天后上述症状无缓解，并出现皮肤、巩膜黄染，伴乏力、腹胀、恶心和纳差，无呕吐、腹痛，无皮肤瘙痒、关节肿痛，无口苦、口干，当地诊所予“头孢呋辛、奥美拉唑”等药物静脉输液，治疗4天后上述症状无缓解。（2020-08-10）于宁夏医科大学总医院查肝功异常：ALP/GGT 273/411U/L, ALT/AST 474/339U/L, TB/DB 134/50umol/L, ALB 29.9g/L，TG 3.18mmol/L；凝血功能：PTA 66.4%，FIB 1.78g/L；嗜肝病毒和非嗜肝病毒感染阴性；腹部超声示：肝脏高回声结节，考虑血管瘤，胆囊炎，脾大，腹水；CT示：肝内小钙化灶；MRCP示：胆囊炎、腹水，Gllisoon鞘膜积液，肝左叶类圆形异常信号灶；考虑诊断“肝损害 药物性肝损害？自身免疫性肝炎”，予“头孢哌酮舒巴坦”抗感染，“舒肝宁、甘草酸单胺半胱氨酸注射液、熊去氧胆酸”保肝、退黄、利胆，补白蛋白、利尿等对症治疗，入院治疗8天后患者未再出现寒战、发热，复查肝功提示胆红素持续升高（最高TB/DB 283/194umol/L）,PTA最低59.06%，口服甲泼尼龙40mg1次后出现低体温（＜35℃），即停药，先后予3次“人工肝治疗+血浆置换术（血浆前予地塞米松5mg）”清除胆红素、炎症因子，后皮肤巩膜黄染较前减轻，复查肝功能好转出院：TB/DB 72/51umol/L, ALP/GGT 133/314U/L, ALT/AST 116/145U/L，凝血功能：PTA 113%。院外口服熊去氧胆酸250mg tid，共20天。13月余前患者自觉皮肤

In [20]:
fewshot_generator(191)

Relevant Context: 入院情况：患者主因“发现肝功异常3年”入院。患者肝功异常主要以ALT、GGT升高为主，保肝治疗GGT呈下降趋势，后无明显诱因逐渐升高。无明显不适症状，平素接触有机物（防护可）。常见铜铁代谢未见异常，抗核抗体谱未见明显异常。 主诉：发现肝功异常3年。 现病史：患者3年前自诉熬夜后体检发现肝功异常：谷丙转氨酶64u/l（未见具体报告）。无食欲下降，无乏力，无腹痛，无尿黄，未予重视。2年前复查肝功仍异常：谷丙转氨酶及谷氨酰转肽酶较前增高（未见报告），于中日友好医院就诊，行甲肝、乙肝、丙肝、戊肝检查未提示异常，诊断“肝功能异常”，给予水飞蓟素治疗。自10月前（2020年8月）起后每月复查一次肝功：ALT波动于30-70u/l。GGT波动于120-200u/l。6月前加用熊去氧胆酸治疗，GGT呈下降趋势，近3月逐渐升高，1月前升高至450u/l，ALT升高至123u/l。2周前于我院门诊就诊，行检查：免疫球蛋白:IgG 1410.0*mg/dl，IgA 285.0*mg/dl，IgM 52.5*mg/dl，铜兰蛋白(CER)0.21*g/l，铁蛋白(Ferritin)164.00*ng/ml，抗核抗体谱:间接免疫荧光法抗核抗体 ()阴性*，间接免疫荧光法双链DNA(dsDNA)阴性*。为进一步诊治收入我科。患者自发病来，神清，精神可，大小便未见明显异常，体重增加8Kg。 既往史: 过敏性鼻炎11年，未予规律诊治。否认高血压、心脏病史，否认糖尿病、脑血管病、精神疾病史。否认肝炎史、结核史、疟疾史。手术史：无，过敏史：有，青霉素类药物过敏。输血史：无，预防接种史：无，传染病史：无。其他系统回顾无特殊。 个人史：出生并久居于本地，否认疫水、疫区接触史，否认其他放射性物质。近3年接触有机物：异丙醇、乙醇、钨类物质。免疫接种史不详。否认吸烟史，否认饮酒史。否认去过疫区、否认接触过从疫区来的人。 家族史：父母健在，否认家族中类似病史、传染病史、遗传病史及肿瘤病史。 检查分类: 超声 检查项目名称: nan 检查部位:nan 检查所见: nan 检查结论: nan 生化检查: 备注-静脉血、标本质量-静脉血、CMV病毒拷贝数-静脉血：<250copies/mL、CMV病毒核酸检测-静脉血：2205
Question: 请使用你的已有知识，判断该病人是以下

In [6]:
print(f"{med.loc[195,'evidence']=}\n")
print(f"{med.loc[195,'context']=}")

med.loc[195,'evidence']='患者中年女性，慢性病程，急性发作；9年前体检发现肝功能异常，转氨酶轻度升高，转肽酶升高明显。腹部超声提示脂肪肝，给予“易善复”口服治疗，肝功能恢复正常，但停药后反复。后规律复查示GGT在300-450U/L、ALT在40-100U/L、AST在30-70U/L、ALP在100-160U/L左右波动。2年前于我院行肝穿刺检查示脂肪肝。出院后给予“易善复”规律治疗。3月前患者体检发现GGT 710U/L。1月前患者复查示GGT 496U/L。根据患者症状、体征、既往史、辅助检查等，患者非酒精性脂肪型肝炎、2型糖尿病、骨质疏松、特纳综合征、甲状腺功能减低诊断明确。'
med.loc[195,'context']='入院情况：患者中年女性，慢性病程，急性发作；9年前体检发现肝功能异常，转氨酶轻度升高，转肽酶升高明显。腹部超声提示脂肪肝，给予“易善复”口服治疗，肝功能恢复正常，但停药后反复。后规律复查示GGT在300-450U/L、ALT在40-100U/L、AST在30-70U/L、ALP在100-160U/L左右波动。2年前于我院行肝穿刺检查示脂肪肝。出院后给予“易善复”规律治疗。3月前患者体检发现GGT 710U/L。1月前患者复查示GGT 496U/L。 主诉：发现肝功能异常9年，加重3月。 现病史：9年前，体检发现肝功能异常，生化示ALT 62U/L，AST 36U/L，ALP 93U/L,GGT 237U/L,TG 2.36mmol/l，TBIL 15.1umol/L，DBIL 3.8umol/L，TP 73g/L，ALB 43g/L。患者自感乏力、多睡，肝区偶有隐痛，无恶心、呕吐、黄疸、发热、关节疼痛等不适，于北京协和医院查彩超提示脂肪肝，给予“易善复”口服治疗，肝功能恢复正常，但停药后反复。后规律复查示GGT在300-450U/L、ALT在40-100U/L、AST在30-70U/L、ALP在100-160U/L左右波动,多次查彩超均提示脂肪肝，查乙肝、丙肝、戊肝标志物阴性，自身免疫性肝病相关检查阴性，未曾行肝脏核磁或CT检查。2年前，于我院行肝穿刺检查示脂肪肝。出院后给予“易善复”规律治疗。3月前，患者体检发现GGT 710U/L，继续口服易善复治疗。1月前，患者复查示GGT 496U/L。现患者为求进一步诊

In [7]:
print(f"{med.loc[198,'evidence']=}")
print(f"{med.loc[198,'context']=}")

med.loc[198,'evidence']='患者青年男性，慢性病程；临床以消化道出血起病，进一步胃镜、影像学检查发现肝硬化合并食管胃底静脉曲张，门静脉系统血栓形成、腹盆腔积液等肝硬化门脉高压表现。病因方面，乙肝、丙肝阴性，免免疫、遗传代谢阴性，肝病理示脂肪性肝炎（F1G2S4)；综合考虑NASH相关肝硬化失代偿诊断明确；因患者反复消化道出血，内镜下治疗效果欠佳，于2022-2-15行颈静脉肝内门体分流术，术后予速碧林序贯利伐沙班抗凝治疗至今，现为术后复查入院。余诊断根据病史可得出。'
med.loc[198,'context']='入院情况：患者主因“反复呕血、黑便3年余，TIPS术后3月”入院。入院查体：神清状可，无肝掌、蜘蛛痣。腹软，无压痛、反跳痛、肌紧张，未触及包块。肝脾未触及。胆囊区无压痛，Murphy氏征阴性。肾脏未触及，肾区及输尿管点无压痛。振水音阴性。肝浊音界正常，肝区、肾区无叩击痛，移动性浊音阴性。肠鸣音正常，未闻及血管杂音，双下肢无水肿。 主诉：反复呕血、黑便3年余，TIPS术后3月 现病史：患者3年余前（2018.11）饱餐后出现黑便，量不大，未予重视，黑便3-4天后出现呕血，为鲜血，量大（具体不详），伴头晕、心慌、乏力，无晕厥、意识丧失、发热，就诊于北京潞河医院，完善相关检查，考虑”肝硬化失代偿期 门脉高压 食管胃底静脉曲张片破裂出血 门静脉及肠系膜上静脉血栓形成、腹盆腔积液 脾大并脾功能亢进”，积极抑酸、止血等内科药物治疗，并行食管胃底静脉曲张EVL+栓塞术。2月后（2019-2）就诊于北京地坛医院，乙肝、丙肝阴性，ENA谱提示NRNAP/Sm阳性反应，Sm阳性反应，于2-19行肝穿刺活检病理示脂肪性肝炎（F1G2S4)；期间复查胃镜示：G-E-2，F2，ESVD(食管胃静脉曲张精准离断术）及EIS（食管静脉硬化治疗）。后不规律服用卡维地洛降门脉高压、水飞蓟素保肝、利可君升白，间断发作黑便、呕血3次，2次于外院行食管胃底静脉曲张EVL+栓塞术，2次内科保守治疗。3月前（2022-2）患者再次因消化道出血住院，完善检查并除外禁忌后，于2022-2-15行颈静脉肝内门体分流术，术后予速碧林序贯利伐沙班抗凝治疗至今，现为术后复查入院。起病以来，患者精神、食欲、睡眠、体力可，大便如上述，小便如常，体重无明显变化。 既往史: 发现门静脉血栓3年余

In [8]:
def process_answer(answer):
    if isinstance(answer,str): return answer
    elif isinstance(answer,list):
        if len(answer)==1: return answer[0]
        else: return '+'.join(answer)
    else: print(answer)
med['answer'].apply(process_answer).unique()

array(['原发性肝癌', '肝囊肿', '肝囊肿+脂肪肝', '肝血管瘤+肝囊肿', '肝血管瘤',
       '原发性肝癌+脂肪肝+肝血管瘤+肝囊肿', '脂肪肝+肝血管瘤', '药物性肝损伤+酒精性肝病+脂肪肝', '酒精性肝病+肝血管瘤',
       '酒精性肝病+脂肪肝', '酒精性肝病', '自身免疫性肝炎', '自身免疫性肝炎+肝囊肿', '脂肪肝+自身免疫性肝炎',
       '脂肪肝+自身免疫性肝炎+药物性肝损伤', '肝血管瘤+自身免疫性肝炎', '肝血管瘤+自身免疫性肝炎+药物性肝损伤',
       '自身免疫性肝炎+药物性肝损伤', '肝囊肿+自身免疫性肝炎', '肝囊肿+自身免疫性肝炎+药物性肝损伤', '药物性肝损伤',
       '肝囊肿+脂肪肝+药物性肝损伤', '肝囊肿+药物性肝损伤', '酒精性肝病+药物性肝损伤', '脂肪肝+药物性肝损伤',
       '自身免疫性肝炎+脂肪肝+药物性肝损伤', '脂肪肝', '脂肪肝+肝囊肿', '药物性肝损伤+脂肪肝',
       '药物性肝损伤+脂肪肝+自身免疫性肝炎'], dtype=object)